In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/temporal')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Extra_and_Unused')
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/"

Running local


In [2]:
# from __future__ import print_function
# from __future__ import print_function
import os
import numpy as np

import datetime
import gc

from scipy import signal

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel # For AE
# from PosAE_training_functions import posautoencoding_m1 as AE, TestModel, TrainModel # For PosAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta

# wandb.login()

cuda


In [3]:
def featuring_autoencoders(dataset, date_format, model, len_features=None, save=True, identifier=None):
    training_loader = DataLoader(dataset, batch_size=1)
    iterator = iter(training_loader)
    testing = TestModel(model, iterator, device=torch.device("cuda"))

    training_recorder_list = []
    training_hour_list = []
    training_minute_list = []
    delete_samples = []
    training_path_samples = []
    training_samples_list_torch = []
    
    if len_features == None:
        len_features = len(training_loader)
    else:
        len_features = len_features
        
    batch = int(len_features*0.1)
    
    for id in range(len_features):
        if (id+1)% batch == 0:
            print(f"id: {id + 1} of {len_features}")
        try:
            originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
        except:
            print(f"error id: {id}")
            delete_samples.append(id)
            continue

    #     encodings_size = encodings[0].shape
        encodings = encodings.to("cuda").detach()
        encodings = encodings.reshape(encodings.shape[0],
                                    encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
        training_samples_list_torch.append(encodings)
        label["recorder"]=np.repeat(label["recorder"][0][0],5)
        training_recorder_list.append(label["recorder"])
        training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
        training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))


        path = np.asarray(path, dtype="U32")
        path = np.repeat(path, 5)
        indexer = ["_1","_2","_3","_4","_5"]
        indexer = np.asarray(indexer)
        # path = path.astype("float64")
        for i in range(len(path)):
#             print(path[i] + identifier[i])
            path[i] = path[i] + indexer[i]
#             print(path[i]) 
        training_path_samples.append(path)
    
#     training_recorder_list = torch.cat(training_recorder_list,dim=0)
    training_hour_list = torch.cat(training_hour_list,dim=0)
    training_minute_list = torch.cat(training_minute_list,dim=0)
    training_samples_list_torch = torch.cat(training_samples_list_torch, dim=0)
    
    if save == True:
    
        if "filters" in dataset.kwargs.keys():
            for value in dataset.filters.values():
                date_format = f"{date_format}_{value}" 
        if identifier != None:
            date_format = f"{date_format}_{identifier}"
        else:
            pass

        torch.save(training_path_samples, f"temporal_zamuro/Features/AE_test_path_samples_{date_format}.pth")
        torch.save(training_samples_list_torch, f"temporal_zamuro/Features/AE_features_{date_format}.pth")
        training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
        torch.save(training_labels_list, f"temporal_zamuro/Features/AE_labels_{date_format}.pth")
        torch.save(delete_samples, f"temporal_zamuro/Features/AE_test_corrupted_samples_list_{date_format}.pth")
        
    else:
        pass

In [4]:
model = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"
model_name = f"{root}/Zamuro/temporal_zamuro/models/model_{model}_{identifier}_{date_format}_final.pth"
config = torch.load(f'{root}/Zamuro/temporal_zamuro/configs/config_{model}_{identifier}_{date_format}.pth', map_location=torch.device('cpu'))
model = AE(num_hiddens=config["num_hiddens"]).to(device)
model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
import os
folders = os.listdir(f"Complementary_Files/Audios_Zamuro/")
# folders = [os.path.splitext(filename)[0] for filename in os.listdir(f"Complementary_Files/Audios_Zamuro/")]

In [6]:
# folders = ["Audios_Jaguas"]
for folder in folders:
    print(folder)
    filters = {"rain_FI": "NO"}
    dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                         dataframe_path=f"Complementary_Files/Audios_Zamuro/{folder}",
                         audio_length=12, ext="wav",
                         win_length=1028, filters=filters)
    featuring_autoencoders(dataset, f"{date_format}_rain", model=model, len_features=None, save=True, identifier=f"{folder}")

RZUD06.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 87 of 871
id: 174 of 871
id: 261 of 871
id: 348 of 871
id: 435 of 871
id: 522 of 871
id: 609 of 871
id: 696 of 871
id: 783 of 871
id: 870 of 871
RZUG04.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 79 of 793
id: 158 of 793
id: 237 of 793
id: 316 of 793
id: 395 of 793
id: 474 of 793
id: 553 of 793
id: 632 of 793
id: 711 of 793
id: 790 of 793
RZUD04.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 84 of 847
id: 168 of 847
id: 252 of 847
id: 336 of 847
id: 420 of 847
id: 504 of 847
id: 588 of 847
id: 672 of 847
id: 756 of 847
id: 840 of 847
RZUE12.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 70 of 703
id: 140 of 703
id: 210 of 703
id: 280 of 703
id: 350 of 703
id: 420 of 703
id: 490 of 703
id: 560 of 703
id: 630 of 703
id: 700 of 703
RZUD09.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 71 of 719


id: 56 of 560
id: 112 of 560
id: 168 of 560
id: 224 of 560
id: 280 of 560
id: 336 of 560
id: 392 of 560
id: 448 of 560
id: 504 of 560
id: 560 of 560
RZUH13.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 73 of 731
id: 146 of 731
id: 219 of 731
id: 292 of 731
id: 365 of 731
id: 438 of 731
id: 511 of 731
id: 584 of 731
id: 657 of 731
id: 730 of 731
RZUB06.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 70 of 703
id: 140 of 703
id: 210 of 703
id: 280 of 703
id: 350 of 703
id: 420 of 703
id: 490 of 703
id: 560 of 703
id: 630 of 703
id: 700 of 703
RZUH06.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 76 of 761
id: 152 of 761
id: 228 of 761
id: 304 of 761
id: 380 of 761
id: 456 of 761
id: 532 of 761
id: 608 of 761
id: 684 of 761
id: 760 of 761
RZUA04.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 60 of 607
id: 120 of 607
id: 180 of 607
id: 240 of 607
id: 300 of 607
id: 360 of 607
id:

id: 279 of 931
id: 372 of 931
id: 465 of 931
id: 558 of 931
id: 651 of 931
id: 744 of 931
id: 837 of 931
id: 930 of 931
RZUA05.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 71 of 716
id: 142 of 716
id: 213 of 716
id: 284 of 716
id: 355 of 716
id: 426 of 716
id: 497 of 716
id: 568 of 716
id: 639 of 716
id: 710 of 716
RZUD12.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 69 of 698
id: 138 of 698
id: 207 of 698
id: 276 of 698
id: 345 of 698
id: 414 of 698
id: 483 of 698
id: 552 of 698
id: 621 of 698
id: 690 of 698
RZUF07.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 76 of 761
id: 152 of 761
id: 228 of 761
id: 304 of 761
id: 380 of 761
id: 456 of 761
id: 532 of 761
id: 608 of 761
id: 684 of 761
id: 760 of 761
RZUH08.csv
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 47 of 472
id: 94 of 472
id: 141 of 472
id: 188 of 472
id: 235 of 472
id: 282 of 472
id: 329 of 472
id: 376 of 472
id:

In [ ]:
filters = {"rain_FI": "NO"}
dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                     dataframe_path="Complementary_Files/zamuro_audios.csv",
                     audio_length=12, ext="wav",
                     win_length=1028, filters=filters)

len_features=None
save=True
identifier=None

training_loader = DataLoader(dataset, batch_size=1)
iterator = iter(training_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))

training_recorder_list = []
training_hour_list = []
training_minute_list = []
delete_samples = []
training_path_samples = []
training_samples_list_torch = []

if len_features == None:
        len_features = len(training_loader)
else:
    len_features = len_features
        
batch = int(len_features*0.1)

for id in range(len_features):
#     if (id+1)%3 == 0:
#         break
    if (id+1)% batch == 0:
        print(f"id: {id + 1} of {len_features}")
        
(originals, reconstructions, encodings, label, loss, path) = testing.reconstruct()
#         except:
#             print(f"error id: {id}")
#             delete_samples.append(id)
#             continue

    #     encodings_size = encodings[0].shape
encodings = encodings.to("cuda").detach()
encodings = encodings.reshape(encodings.shape[0],
                            encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
training_samples_list_torch.append(encodings)
label["recorder"]=np.repeat(label["recorder"][0][0],5)
training_recorder_list.append(label["recorder"])
training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))



In [ ]:
model = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"
y = torch.load(f"temporal_zamuro/Features/AE_labels_{date_format}_rain_NO_RZUA02.csv.pth",  map_location=torch.device('cpu'))
X = torch.load(f"temporal_zamuro/Features/AE_features_{date_format}_rain_NO_RZUA02.csv.pth",  map_location=torch.device('cpu'))

In [ ]:
muestra = X
muestra[0:5]

In [ ]:
model_type = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 9
hour = 4
month = 6
folder = "AE_rain"
date_format = f"day_{day}_hour_{hour}"
y = torch.load(f"temporal_zamuro/Features/{folder}/AE_labels_{date_format}_rain_NO_RZUA01.pth",  map_location=torch.device('cpu'))
X = torch.load(f"temporal_zamuro/Features/{folder}/AE_features_{date_format}_rain_NO_RZUA01.pth",  map_location=torch.device('cpu'))
path = torch.load(f"temporal_zamuro/Features/{folder}/AE_test_path_samples_{date_format}_rain_NO_RZUA01.pth",  map_location=torch.device('cpu'))
path_flat = [item for sublist in path for item in sublist]
path_flat = np.asarray(path_flat)